In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/data/webmd.csv")
train

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1
...,...,...,...,...,...,...,...,...,...,...,...,...
362801,55-64,Stop Smoking,11/14/2008,chantix,144470,5,5,I took the whole 12 weeks.I could have stopped...,5,Female,"Nausea , headache , vomiting , drowsiness, g...",2
362802,,Other,11/14/2008,chantix,144470,2,3,My dad strated taking Chantix about 1 month ag...,1,,"Nausea , headache , vomiting , drowsiness, g...",16
362803,25-34,Stop Smoking,11/13/2008,chantix,144470,2,4,"As long as I was on chantix, I didn't smoke. ...",2,Female,"Nausea , headache , vomiting , drowsiness, g...",3
362804,55-64,Stop Smoking,11/13/2008,chantix,144470,5,5,Started this medication Oct 5th 2008. Haven't ...,5,Male,"Nausea , headache , vomiting , drowsiness, g...",1


In [ ]:
def tokenizer(sent):
    tokenizer_initial = TreebankWordTokenizer()
    return tokenizer_initial.tokenize(sent)

def rm_stopword(tok):
    stop = stopwords.words('english')
    return [i for i in tok if i not in stop]

def stem(tok):
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(i) for i in tok])

def preprocess(data):
    # Converting to lowercase
    data['review_clean'] = data['Reviews'].str.lower()

    # Remove links
    data['review_clean'] = data['review_clean'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

    # Remove punctuations
    data['review_clean'] = data['review_clean'].str.replace('[^\w\s]','')

    # Remove Numbers
    data['review_clean'] = data['review_clean'].str.replace('\d+', '')

    # Tokenizing
    data['review_clean'] = data['review_clean'].apply(str).apply(lambda x: tokenizer(x))

    # Remove stopwords
    data['review_clean'] = data['review_clean'].apply(lambda x: rm_stopword(x))

    # Stemming
    data['review_clean'] = data['review_clean'].apply(lambda x: stem(x))

    return data

In [ ]:
df = preprocess(train)
df

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount,review_clean
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0,im retir physician med tri allergi season one ...
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1,clear right even throat hurt went away take me...
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0,ptinr go normal
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0,fall dont realis
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1,grandfath prescrib medic coumadin assist blood...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362801,55-64,Stop Smoking,11/14/2008,chantix,144470,5,5,I took the whole 12 weeks.I could have stopped...,5,Female,"Nausea , headache , vomiting , drowsiness, g...",2,took whole weeksi could stop drug sooner thoug...
362802,,Other,11/14/2008,chantix,144470,2,3,My dad strated taking Chantix about 1 month ag...,1,,"Nausea , headache , vomiting , drowsiness, g...",16,dad strate take chantix month ago fine stop sm...
362803,25-34,Stop Smoking,11/13/2008,chantix,144470,2,4,"As long as I was on chantix, I didn't smoke. ...",2,Female,"Nausea , headache , vomiting , drowsiness, g...",3,long chantix didnt smoke terribl stomach upset...
362804,55-64,Stop Smoking,11/13/2008,chantix,144470,5,5,Started this medication Oct 5th 2008. Haven't ...,5,Male,"Nausea , headache , vomiting , drowsiness, g...",1,start medic oct th havent smoke week plan full...


In [ ]:
df.DrugId.value_counts()

6873      8538
1025      7868
6007      7728
4398      5864
14208     5451
          ... 
78479        1
14766        1
165950       1
8244         1
18441        1
Name: DrugId, Length: 6572, dtype: int64

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['review_clean'])

X = df['review_clean']
# y = df['DrugId']
# y_prime = y.apply(lambda x: x-1)
# y_cat = to_categorical(y_prime)

X_seq = tokenizer.texts_to_sequences(X)

max_len = 25
X_pad = pad_sequences(X_seq, maxlen = max_len)
# X_train, X_test, y_train, y_test = train_test_split(X, y_cat, stratify = y_cat, test_size = 0.2, random_state = 42)

# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)

# max_len = 25
# X_train = pad_sequences(X_train, maxlen = max_len)
# X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-12-03 03:20:31--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.99.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.99.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  46.4MB/s    in 36s     

2021-12-03 03:21:07 (43.9 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
from gensim.models import KeyedVectors

EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary = True)

In [ ]:
embedding_dim = 300
# unique_vocab = len(tokenizer.word_index)
unique_vocab = 150000
embedding_matrix = np.zeros((unique_vocab + 1, embedding_dim))

for word, i in tokenizer.word_index.items():      
    if word in word_vectors.vocab:
        embedding_matrix[i] = word_vectors[word]

In [ ]:
from keras.models import load_model

model = load_model("/content/drive/MyDrive/data/drug_review_model3.h5")

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 25, 300)           45000300  
                                                                 
 conv1d_12 (Conv1D)          (None, 25, 300)           450300    
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 12, 300)          0         
 g1D)                                                            
                                                                 
 batch_normalization_12 (Bat  (None, 12, 300)          1200      
 chNormalization)                                                
                                                                 
 spatial_dropout1d_12 (Spati  (None, 12, 300)          0         
 alDropout1D)                                                    
                                                      

In [ ]:
unique_vocab

140743

In [ ]:
embedding_matrix.shape

(140744, 300)

In [ ]:
X_pad.shape

(362806, 25)

In [ ]:
df.DrugId.value_counts()

6873      8538
1025      7868
6007      7728
4398      5864
14208     5451
          ... 
78479        1
14766        1
165950       1
8244         1
18441        1
Name: DrugId, Length: 6572, dtype: int64

In [ ]:
y_pred1 = model.predict(X_pad)

In [ ]:
y_pred1

array([[2.0193605e-01, 1.5254606e-02, 1.2643500e-02, ..., 1.9451546e-02,
        5.2346524e-02, 6.6933328e-01],
       [2.7916320e-02, 8.0642104e-03, 1.2127081e-02, ..., 1.2685438e-01,
        2.5240716e-01, 4.7231323e-01],
       [5.3577695e-02, 7.5380504e-03, 9.0489825e-03, ..., 5.2047268e-02,
        1.3585401e-01, 6.9673634e-01],
       ...,
       [8.8974256e-03, 7.0104311e-04, 1.0288018e-03, ..., 3.0024046e-02,
        1.4186361e-01, 8.0729252e-01],
       [1.3010863e-02, 7.8047463e-04, 1.0123578e-03, ..., 1.9100534e-02,
        9.6370980e-02, 8.6235428e-01],
       [1.0485083e-02, 1.2024285e-02, 2.2373142e-02, ..., 2.9790834e-01,
        2.5404271e-01, 1.1455285e-01]], dtype=float32)

In [ ]:
df['rating'] = np.argmax(y_pred1, axis = 1)

In [ ]:
df

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount,review_clean,rating
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0,im retir physician med tri allergi season one ...,9
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1,clear right even throat hurt went away take me...,9
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0,ptinr go normal,9
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0,fall dont realis,0
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1,grandfath prescrib medic coumadin assist blood...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362801,55-64,Stop Smoking,11/14/2008,chantix,144470,5,5,I took the whole 12 weeks.I could have stopped...,5,Female,"Nausea , headache , vomiting , drowsiness, g...",2,took whole weeksi could stop drug sooner thoug...,9
362802,,Other,11/14/2008,chantix,144470,2,3,My dad strated taking Chantix about 1 month ag...,1,,"Nausea , headache , vomiting , drowsiness, g...",16,dad strate take chantix month ago fine stop sm...,0
362803,25-34,Stop Smoking,11/13/2008,chantix,144470,2,4,"As long as I was on chantix, I didn't smoke. ...",2,Female,"Nausea , headache , vomiting , drowsiness, g...",3,long chantix didnt smoke terribl stomach upset...,9
362804,55-64,Stop Smoking,11/13/2008,chantix,144470,5,5,Started this medication Oct 5th 2008. Haven't ...,5,Male,"Nausea , headache , vomiting , drowsiness, g...",1,start medic oct th havent smoke week plan full...,9


In [ ]:
df.to_csv("/content/drive/MyDrive/data/webmd_rating.csv", index = False)

In [ ]:
df_loaded = pd.read_csv("/content/drive/MyDrive/data/webmd_rating.csv")
df_loaded

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount,review_clean,rating
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0,im retir physician med tri allergi season one ...,9
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1,clear right even throat hurt went away take me...,9
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0,ptinr go normal,9
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0,fall dont realis,0
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1,grandfath prescrib medic coumadin assist blood...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362801,55-64,Stop Smoking,11/14/2008,chantix,144470,5,5,I took the whole 12 weeks.I could have stopped...,5,Female,"Nausea , headache , vomiting , drowsiness, g...",2,took whole weeksi could stop drug sooner thoug...,9
362802,,Other,11/14/2008,chantix,144470,2,3,My dad strated taking Chantix about 1 month ag...,1,,"Nausea , headache , vomiting , drowsiness, g...",16,dad strate take chantix month ago fine stop sm...,0
362803,25-34,Stop Smoking,11/13/2008,chantix,144470,2,4,"As long as I was on chantix, I didn't smoke. ...",2,Female,"Nausea , headache , vomiting , drowsiness, g...",3,long chantix didnt smoke terribl stomach upset...,9
362804,55-64,Stop Smoking,11/13/2008,chantix,144470,5,5,Started this medication Oct 5th 2008. Haven't ...,5,Male,"Nausea , headache , vomiting , drowsiness, g...",1,start medic oct th havent smoke week plan full...,9
